# Testing odometry

In [ ]:
import torch
from torch.utils.data import DataLoader
from slam_framework.neural_slam import NeuralSLAM
from helpers import Arguments, log
from odometry.vo_datasets import KittiOdometryDataset
import time
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100


args = Arguments.get_arguments()
sequence_length = 1
preprocessed_flow = True

dataset = KittiOdometryDataset(args.data_path, "00", precomputed_flow=preprocessed_flow, sequence_length=sequence_length)
slam = NeuralSLAM(args, preprocessed_flow=preprocessed_flow)

In [ ]:
slam.start_odometry()
print("SLAM mode: ", slam.mode())

In [ ]:
global_scale = []
count = 0
slam_call_time = []
for i in range(0, len(dataset), sequence_length):
    if preprocessed_flow:
        img1, img2, flow, _, __ = dataset[i]
        start = time.time()
        current_pose = slam(img1.squeeze(), img2.squeeze(), flow.squeeze())
        end = time.time()
    else:
        img1, img2, _, __ = dataset[i]
        start = time.time()
        current_pose = slam(img1.squeeze(), img2.squeeze())
        end = time.time()
        
    slam_call_time.append(end-start)
    global_scale += current_pose
    
    print('    ', end='\r')
    print(count, end='')
    count = count+sequence_length

global_scale = torch.stack(global_scale, dim=0)
slam_call_time = np.array(slam_call_time)

## FPS calculation

In [ ]:
# Numpy time
log("Average odometry time: ", slam_call_time.mean())
log("Odometry time std: ", slam_call_time.std())

# FPS manual calc
fps_manual = 1/(slam_call_time.mean())
log("FPS from time: ", 1/slam_call_time.mean())

## Keyframe check

In [ ]:
slam.end_odometry()

In [ ]:
import matplotlib.pyplot as plt
DATA_PATH = args.keyframes_path + "/poses.pth"
poses = torch.load(DATA_PATH)
print(poses.shape)
X = poses[:, 3]
Z = poses[:, -1]
plt.scatter(X.numpy(), Z.numpy())
plt.show()

In [ ]:
keyframe_poses = []
for i in range(len(slam)):
    keyframe_poses.append(slam.get_keyframe(i).pose)

In [ ]:
keyframe_positions = []
for pose in keyframe_poses:

    keyframe_positions.append(pose[:-1, -1])

keyframe_positions = torch.stack(keyframe_positions, dim=0)
print(keyframe_positions.shape)

In [ ]:
import matplotlib.pyplot as plt
#import matplotlib as mpl
#mpl.rcParams['figure.dpi'] = 150

global_pos = global_scale[:, :3, -1]
X, Y, Z = global_pos[:, 0], global_pos[:, 1], global_pos[:, 2]
X_key, Y_key, Z_key = keyframe_positions[:, 0], keyframe_positions[:, 1], keyframe_positions[:, 2]

plt.plot(X, Z)
#plt.plot(X[:100], Z[:100])
plt.scatter(X_key, Z_key)
plt.show()

plt.plot(X)
plt.show()

plt.plot(Y)
plt.show()

plt.plot(Z)
plt.show()

# Testing mapping

In [ ]:
import torch
from torch.utils.data import DataLoader
from slam_framework.neural_slam import NeuralSLAM
from helpers import Arguments, log
from odometry.vo_datasets import KittiOdometryDataset
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

args = Arguments()
sequence_length = 1
preprocessed_flow = True
DATA_PATH = None # TODO Change to config file data read
slam = NeuralSLAM(DATA_PATH, preprocessed_flow=preprocessed_flow, start_mode="relocalization")

In [ ]:
from localization.localization_dataset import MappingDataset, KittiLocalizationDataset
from GMA.core.utils.utils import InputPadder
from helpers import log, matrix2euler
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

dataset = MappingDataset(args.keyframes_path, slam=True)
rgb_mean = torch.load("normalization_cache/rgb_mean.pth").unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
rgb_std = torch.load("normalization_cache/rgb_std.pth").unsqueeze(-1).unsqueeze(-1).unsqueeze(0)

dataset = KittiLocalizationDataset(data_path=args.data_path, sequence="00")

rgb, true_orientation, true_position = dataset[195]

padder = InputPadder(rgb.shape)
rgb = padder.pad(rgb)[0]

im_normalized = (rgb.unsqueeze(0)-rgb_mean)/rgb_std

with torch.no_grad():
    initial_pose, refined_pose, distances = slam(im_normalized)
    #log("Distances shape", distances.shape)

    max_dist = torch.max(distances).cpu().numpy()
    bins = 1000

    [hist, bin_edges] = np.histogram(distances.cpu().numpy(), bins=bins)

    # ---------
    # Histogram
    # ---------

    plt.bar(bin_edges[:-1], hist, width=5)
    plt.xlabel("Distance from sample")
    plt.ylabel("Count of elements")
    plt.show()

    # Predicted index
    distances_mean = distances.mean()
    pred_index = torch.argmin(distances)
    second_pred_index = torch.argmin(distances)

    plt.plot(distances.cpu().numpy())
    plt.xlabel("Index of keyframe")
    plt.ylabel("Embedding distance from sample")
    plt.show()

def prepare_im(im):
    return im.detach().byte().squeeze().permute(1, 2, 0).numpy()


pred_im, _, _ = dataset[int(pred_index.squeeze())]
second_pred_im, _, _ = dataset[int(second_pred_index.squeeze())]

def to_vectors(mat):
    abs_rotation = mat[:3, :3]
    abs_translation = mat[:3, -1]

    orientation = matrix2euler(abs_rotation)
    position = abs_translation
    return orientation, position

initial_orientation, initial_position = to_vectors(initial_pose)
refined_orientation, refined_position = to_vectors(refined_pose)
log("True pose: ", [true_orientation, true_position])
log("Initial estimate: ", [initial_orientation, initial_position])
log("Refined estimate: ", [refined_orientation, refined_position])

log("Initial difference: ", [(true_orientation-initial_orientation).abs().sum(), (true_position-initial_position).abs().sum()])
log("Refined difference: ", [(true_orientation-refined_orientation).abs().sum(), (true_position-refined_position).abs().sum()])

In [ ]:
DATA_PATH = args.data_path + "/dataset/poses/00.txt"
true_pos = np.loadtxt(DATA_PATH)

X_gt, Y_gt, Z_gt = true_pos[:, 3], true_pos[:, 7], true_pos[:, 11]

index = 2400
diff = 20
plt.plot(X_gt, Z_gt)
plt.plot(X_gt[index-diff:index+diff], Z_gt[index-diff:index+diff])
plt.show()

## Testing reconstruction of keyframes

In [ ]:
poses = []
for i in range(len(slam)):
    poses.append(slam[i].pose.flatten())
poses = torch.stack(poses, dim=0)

X = poses[:, 3]
Z = poses[:, 11]

index = 195
plt.scatter(X, Z)
plt.scatter(X[index], Z[index])
plt.show()

## Reconstructing image from path

In [ ]:
rgb = torch.load(slam[100].rgb_file_name)
rgb_mean = torch.load("normalization_cache/rgb_mean.pth").unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
rgb_std = torch.load("normalization_cache/rgb_std.pth").unsqueeze(-1).unsqueeze(-1).unsqueeze(0)

rgb = (rgb*rgb_std)+rgb_mean

print(rgb.shape)
plt.imshow(rgb.squeeze().byte().permute(1, 2, 0).numpy())
plt.show()